In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
from sklearn import preprocessing

In [2]:
data_dir = Path.cwd().parent / 'DATA'

In [3]:
# define target column
TARGET = "contest-tmp2m-14d__tmp2m"

# 0. Load Train/Test Data

In [4]:
%%time
train_df = pd.read_parquet(data_dir / "train_data.parquet")
print(f"training dataframe -> rows: {train_df.shape[0]}, cols: {train_df.shape[1]}")
train_mem_size = train_df.memory_usage(deep=True).sum() / 1024
print(f"-- memory usage: {round(train_mem_size):,} MB")

training dataframe -> rows: 375734, cols: 246
-- memory usage: 761,726 MB
CPU times: total: 2.03 s
Wall time: 378 ms


In [5]:
%%time
test_df = pd.read_parquet(data_dir / "test_data.parquet")
print(f"training dataframe -> rows: {test_df.shape[0]}, cols: {test_df.shape[1]}")
test_mem_size = test_df.memory_usage(deep=True).sum() / 1024
print(f"-- memory usage: {round(test_mem_size):,} MB")

training dataframe -> rows: 31354, cols: 245
-- memory usage: 63,342 MB
CPU times: total: 31.2 ms
Wall time: 60.6 ms


# 1. Address Data Quality Issues

## a. Location Mismatch between Train & Test

ref: <https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved>

In [6]:
SCALE = 14

In [7]:
def fix_locations(in_df: pd.DataFrame) -> pd.DataFrame:
    for col in ["lat", "lon"]:
        in_df.loc[:, col] = round(train_df[col], SCALE)
    return in_df

In [8]:
train1_df = fix_locations(in_df=train_df)
test1_df = fix_locations(in_df=test_df)
train_df, test_df = "", ""

In [9]:
# Concatenate train and test data
all_df = pd.concat([train1_df, test1_df], axis=0)

# Create new feature
all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()

print(f'{all_df.loc_group.nunique()} unique locations')

514 unique locations


## b. Fill Missing Values w/ Mean

In [12]:
def fill_missing(in_df: pd.DataFrame) -> pd.DataFrame:
    out_df = in_df.copy()
    cols_w_na = in_df.columns[in_df.isna().any()].to_list()
    if len(cols_w_na):
        print("columns with missing values:\n")
        count_tot = out_df.shape[0]
        for col in cols_w_na:
            count_na = out_df[col].isna().sum()
            print(f"{col} -> count missing: {count_na}, percent: {round(count_na / count_tot * 100, 2)}%")
            col_mean = out_df[col].mean()
            print(f"-- mean: {round(col_mean, 4)}\n")
            out_df[col] = out_df[col].fillna(col_mean)
    else:
        print("no columns w/ missing values")
    return out_df

In [13]:
train2_df = fill_missing(in_df=train1_df)

columns with missing values:

nmme0-tmp2m-34w__ccsm30 -> count missing: 15934, percent: 4.24%
-- mean: 10.0993

nmme-tmp2m-56w__ccsm3 -> count missing: 10280, percent: 2.74%
-- mean: 9.6632

nmme-prate-34w__ccsm3 -> count missing: 8738, percent: 2.33%
-- mean: 31.2615

nmme0-prate-56w__ccsm30 -> count missing: 15934, percent: 4.24%
-- mean: 29.624

nmme0-prate-34w__ccsm30 -> count missing: 15934, percent: 4.24%
-- mean: 29.6526

nmme-prate-56w__ccsm3 -> count missing: 10280, percent: 2.74%
-- mean: 32.0076

nmme-tmp2m-34w__ccsm3 -> count missing: 8738, percent: 2.33%
-- mean: 9.821

ccsm30 -> count missing: 15934, percent: 4.24%
-- mean: 10.1015



In [14]:
test2_df = fill_missing(in_df=test1_df)

no columns w/ missing values


In [15]:
train1_df, test1_df = "", ""

# 2. Transform Data

## a. Determine Columns w/ High Correlation

In [16]:
THRSHLD = 0.9

In [17]:
skip_cols = [TARGET, "index"]

def column_corr(in_df: pd.DataFrame) -> list:
    trgt_corr = in_df.corr(numeric_only=True)[TARGET].sort_values(ascending=False)
    all_cols = trgt_corr.index.to_list()
    keep_cols = [c for c in all_cols if c not in skip_cols]
    trgt_corr = trgt_corr[trgt_corr.index.isin(keep_cols)]
    trgt_corr.name = "correlation"
    corr_df = pd.DataFrame(trgt_corr)
    corr_df["abs. corr."] = corr_df["correlation"].abs()
    top_corr_df = corr_df[corr_df["abs. corr."] >= THRSHLD]
    print(f"columns w/ abs. corr. above {THRSHLD}: {top_corr_df.shape[0]}")
    display(top_corr_df)
    return top_corr_df.index.to_list()

In [18]:
%%time
top_corr_cols = column_corr(in_df=train2_df)

columns w/ abs. corr. above 0.9: 19


,correlation,abs. corr.
nmme-tmp2m-56w__cfsv2,0.954668,0.954668
nmme-tmp2m-34w__cfsv2,0.954483,0.954483
nmme-tmp2m-56w__nmmemean,0.950865,0.950865
nmme-tmp2m-34w__nmmemean,0.950187,0.950187
nmme-tmp2m-56w__gfdlflora,0.949846,0.949846
nmme-tmp2m-34w__gfdlflorb,0.949346,0.949346
nmme-tmp2m-56w__gfdlflorb,0.949016,0.949016
nmme-tmp2m-34w__gfdlflora,0.947379,0.947379
nmme-tmp2m-56w__ccsm4,0.944435,0.944435
nmme-tmp2m-34w__ccsm4,0.943234,0.943234


CPU times: total: 34.5 s
Wall time: 45.8 s


In [26]:
baseline_cols = ["startdate", "lat", "lon"]
keep_test = baseline_cols + top_corr_cols
keep_train = keep_test + [TARGET]

In [27]:
train3_df = train2_df[keep_train].copy()
test3_df = test2_df[keep_test].copy()
train2_df, test2_df = "", ""

In [32]:
print(f"train3 dataframe -> rows: {train3_df.shape[0]}, cols: {train3_df.shape[1]}")
print(f"test3 dataframe -> rows: {test3_df.shape[0]}, cols: {test3_df.shape[1]}")

train3 dataframe -> rows: 375734, cols: 26
test3 dataframe -> rows: 31354, cols: 25


## b. Create Date Features

In [33]:
def prep_features(in_df: pd.DataFrame) -> pd.DataFrame:
    in_df["date"] = pd.to_datetime(in_df["startdate"])
    in_df["week"] = in_df["date"].dt.isocalendar().week
    in_df["month"] = in_df["date"].dt.month
    print(in_df[["date", "week", "month"]].head())
    return in_df.drop(columns=["startdate"])

In [34]:
%%time
train4_df = prep_features(in_df=train3_df)

        date  week  month
0 2014-09-01    36      9
1 2014-09-02    36      9
2 2014-09-03    36      9
3 2014-09-04    36      9
4 2014-09-05    36      9
CPU times: total: 15.8 s
Wall time: 19.5 s


In [35]:
%%time
test4_df = prep_features(in_df=test3_df)

        date  week  month
0 2022-11-01    44     11
1 2022-11-02    44     11
2 2022-11-03    44     11
3 2022-11-04    44     11
4 2022-11-05    44     11
CPU times: total: 15.6 ms
Wall time: 24.4 ms


In [36]:
train3_df, test3_df = "", ""

In [59]:
train4_df.dtypes

lat                                 float64
lon                                 float64
nmme-tmp2m-56w__cfsv2               float64
nmme-tmp2m-34w__cfsv2               float64
nmme-tmp2m-56w__nmmemean            float64
nmme-tmp2m-34w__nmmemean            float64
nmme-tmp2m-56w__gfdlflora           float64
nmme-tmp2m-34w__gfdlflorb           float64
nmme-tmp2m-56w__gfdlflorb           float64
nmme-tmp2m-34w__gfdlflora           float64
nmme-tmp2m-56w__ccsm4               float64
nmme-tmp2m-34w__ccsm4               float64
nmme-tmp2m-56w__nasa                float64
nmme-tmp2m-34w__nasa                float64
nmme-tmp2m-56w__gfdl                float64
nmme-tmp2m-34w__gfdl                float64
nmme-tmp2m-34w__cancm3              float64
nmme-tmp2m-56w__cancm3              float64
nmme-tmp2m-34w__cancm4              float64
nmme-tmp2m-56w__cancm4              float64
nmme-tmp2m-56w__ccsm3               float64
contest-tmp2m-14d__tmp2m            float64
date                         dat

## c. Scale the Features

In [55]:
scaler = preprocessing.MinMaxScaler()

In [56]:
scaled_train_cols = ["week", "month", TARGET] + top_corr_cols
scaled_test_cols = ["week", "month"] + top_corr_cols
cols_to_skip = ["lat", "lon"]

In [69]:
def scale_features(in_df: pd.DataFrame, cols_to_scale: list) -> pd.DataFrame:
    slct_df = in_df[cols_to_scale]
    scaler.fit(slct_df)
    scaled_ndarray = scaler.transform(slct_df)
    out_df = pd.DataFrame(scaled_ndarray)
    out_df.columns = scaled_cols
    out_df[cols_to_skip] = in_df[cols_to_skip]
    out_df.index = in_df["date"]
    return out_df

In [70]:
train_scaled = scale_features(in_df=train4_df, )
train_scaled.head()

,week,month,contest-tmp2m-14d__tmp2m,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-56w__nmmemean,nmme-tmp2m-34w__nmmemean,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-56w__gfdlflorb,...,nmme-tmp2m-34w__nasa,nmme-tmp2m-56w__gfdl,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__cancm3,nmme-tmp2m-56w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,lat,lon
date,,,,,,,,,,,,,,,,,,,,,
2014-09-01,0.673077,0.727273,0.852524,0.836139,0.827369,0.860309,0.858386,0.803415,0.817825,0.832576,...,0.841984,0.842889,0.837906,0.834871,0.842156,0.824174,0.823033,0.882232,0.0,0.833333
2014-09-02,0.673077,0.727273,0.846032,0.836139,0.827369,0.860309,0.858386,0.803415,0.817825,0.832576,...,0.841984,0.842889,0.837906,0.834871,0.842156,0.824174,0.823033,0.882232,0.0,0.833333
2014-09-03,0.673077,0.727273,0.841908,0.836139,0.827369,0.860309,0.858386,0.803415,0.817825,0.832576,...,0.841984,0.842889,0.837906,0.834871,0.842156,0.824174,0.823033,0.882232,0.0,0.833333
2014-09-04,0.673077,0.727273,0.844057,0.836139,0.827369,0.860309,0.858386,0.803415,0.817825,0.832576,...,0.841984,0.842889,0.837906,0.834871,0.842156,0.824174,0.823033,0.882232,0.0,0.833333
2014-09-05,0.673077,0.727273,0.846063,0.836139,0.827369,0.860309,0.858386,0.803415,0.817825,0.832576,...,0.841984,0.842889,0.837906,0.834871,0.842156,0.824174,0.823033,0.882232,0.0,0.833333
